In [2]:
from blitz_env.simulate_draft_sqlite import simulate_draft, visualize_draft_board
from blitz_env.models import DatabaseManager, Player

def draft_player() -> str:
    db = DatabaseManager()
    try:
        # First, try to get the best available QB
        best_qb = db.session.query(Player).filter(
            Player.availability == 'AVAILABLE',
            Player.allowed_positions.contains("QB"),
        ).order_by(Player.rank).first()
        
        if best_qb:
            return best_qb.id
        
        # If no QBs available, get best overall player
        best_player = db.session.query(Player).filter(
            Player.availability == 'AVAILABLE'
        ).order_by(Player.rank).first()
        
        return best_player.id if best_player else ""
    finally:
        db.close()

# Run the draft simulation - data is stored in SQLite database
simulate_draft(draft_player, 2025)

OperationalError: (sqlite3.OperationalError) table league_settings has no column named num_teams
[SQL: INSERT INTO league_settings (year, player_slots, is_snake_draft, total_rounds, points_per_reception, num_teams) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: (2025, '{"QB": 2, "RB": 2, "WR": 2, "TE": 1, "FLEX": 1, "K": 1, "DST": 1, "Bench": 3}', 1, 15, 1.0, 10)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [3]:
visualize_draft_board()

OperationalError: (sqlite3.OperationalError) no such column: league_settings.num_teams
[SQL: SELECT league_settings.id AS league_settings_id, league_settings.year AS league_settings_year, league_settings.player_slots AS league_settings_player_slots, league_settings.is_snake_draft AS league_settings_is_snake_draft, league_settings.total_rounds AS league_settings_total_rounds, league_settings.points_per_reception AS league_settings_points_per_reception, league_settings.num_teams AS league_settings_num_teams 
FROM league_settings
 LIMIT ? OFFSET ?]
[parameters: (1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)